In [1]:
import json
import glob
from pyroengine.vision import Classifier
import argparse
import glob
import io
import json
import sys
from pathlib import Path
from typing import List, Tuple
import json
import numpy as np
import requests
from PIL import Image
from tqdm import tqdm


In [2]:
model = Classifier()

In [3]:
json_files = glob.glob("data/demo77/**/response*.json")
len(json_files)

3

In [4]:
def format_bboxes(arr: np.ndarray) -> str:
    if arr.size == 0:
        return "[]"
    rounded = np.round(arr.astype(float), 3)
    parts = []
    for row in rounded:
        x1, y1, x2, y2, s = row.tolist()
        parts.append(f"({x1:.3f},{y1:.3f},{x2:.3f},{y2:.3f},{s:.3f})")
    return "[" + ",".join(parts) + "]"

def download_image(url: str, timeout: int = 30) -> Image.Image:
    resp = requests.get(url, timeout=timeout)
    resp.raise_for_status()
    return Image.open(io.BytesIO(resp.content)).convert("RGB")

def process_file(json_path: Path, replace_when_empty: bool = False) -> Path | None:
    data = json.loads(json_path.read_text())
    modified = False

    for entry in tqdm(data, desc=f"{json_path.name}", leave=False):
        url = entry.get("url")
        if not url:
            continue
        try:
            im = download_image(url)
            pred = model(im)   # call your Classifier directly
            if not isinstance(pred, np.ndarray):
                pred = np.array(pred, dtype=float)

            if pred.ndim == 2 and pred.shape[1] == 5:
                if pred.shape[0] > 0 or replace_when_empty:
                    entry["bboxes"] = format_bboxes(pred)
                    modified = True
        except Exception as e:
            print(f"[WARN] {url} failed: {e}")
            continue

    if modified:
        out_path = json_path.parent / "response.json"
        out_path.write_text(json.dumps(data, ensure_ascii=False, indent=2))
        return out_path
    return None


In [5]:
# Process all response*.json files
json_files = glob.glob("data/demo77/**/response*.json", recursive=True)
print(f"Found {len(json_files)} files")

for p in tqdm(json_files, desc="Files"):
    out = process_file(Path(p), replace_when_empty=False)
    if out:
        print(f"Saved updated file to {out}")

Found 3 files


Files:   0%|          | 0/3 [00:00<?, ?it/s]2025-09-17 07:55:53,861 | INFO: Model original pred : []
2025-09-17 07:55:54,440 | INFO: Model original pred : [[3.0517578e-05 5.2821183e-01 1.7272949e-02 5.5164933e-01 4.0258789e-01]]
2025-09-17 07:55:54,905 | INFO: Model original pred : [[3.0517578e-05 5.1171875e-01 2.2766113e-02 5.5078125e-01 1.7480469e-01]]
2025-09-17 07:55:55,325 | INFO: Model original pred : [[0.         0.5234375  0.02085876 0.5512153  0.36938477]]
2025-09-17 07:55:56,393 | INFO: Model original pred : []
2025-09-17 07:55:57,101 | INFO: Model original pred : [[9.1552734e-05 5.3081596e-01 1.6601562e-02 5.5251735e-01 1.5600586e-01]]
2025-09-17 07:55:58,063 | INFO: Model original pred : []
2025-09-17 07:55:59,493 | INFO: Model original pred : []
2025-09-17 07:56:03,268 | INFO: Model original pred : [[0.         0.5277778  0.02450562 0.5503472  0.41552734]]
2025-09-17 07:56:04,938 | INFO: Model original pred : [[0.00105286 0.52821183 0.02548218 0.55164933 0.33129883]]
Files

Saved updated file to data/demo77/croix-augas_-_190-13-24/response.json


2025-09-17 07:56:05,956 | INFO: Model original pred : []
2025-09-17 07:56:06,751 | INFO: Model original pred : []
2025-09-17 07:56:07,127 | INFO: Model original pred : [[0.17126465 0.43511283 0.17932129 0.44856772 0.26464844]]
2025-09-17 07:56:07,561 | INFO: Model original pred : [[0.1854248  0.4309896  0.19470215 0.4483507  0.3642578 ]]
2025-09-17 07:56:07,945 | INFO: Model original pred : [[0.16046143 0.4327257  0.17156982 0.4500868  0.19592285]]
2025-09-17 07:56:08,279 | INFO: Model original pred : [[0.16326904 0.43381077 0.17803955 0.44986978 0.16125488]]
2025-09-17 07:56:08,623 | INFO: Model original pred : [[0.17565918 0.43229166 0.18688965 0.4487847  0.5917969 ]]
2025-09-17 07:56:08,943 | INFO: Model original pred : [[0.18414307 0.42578125 0.199646   0.4483507  0.6352539 ]]
2025-09-17 07:56:09,301 | INFO: Model original pred : [[0.15472412 0.43337673 0.18658447 0.45203993 0.25463867]]
2025-09-17 07:56:09,774 | INFO: Model original pred : [[0.15478516 0.42144096 0.17602539 0.4509

Saved updated file to data/demo77/nemours_-_25-13-24/response.json


2025-09-17 07:56:10,183 | INFO: Model original pred : []
2025-09-17 07:56:10,567 | INFO: Model original pred : []
2025-09-17 07:56:10,990 | INFO: Model original pred : []
2025-09-17 07:56:11,297 | INFO: Model original pred : []
2025-09-17 07:56:11,732 | INFO: Model original pred : []
2025-09-17 07:56:12,055 | INFO: Model original pred : [[0.38830566 0.4626736  0.42126465 0.49739584 0.31445312]]
2025-09-17 07:56:12,477 | INFO: Model original pred : [[0.39794922 0.4704861  0.43603516 0.49913195 0.40405273]]
2025-09-17 07:56:12,796 | INFO: Model original pred : [[0.39672852 0.46875    0.4099121  0.49479166 0.15283203]]
2025-09-17 07:56:13,169 | INFO: Model original pred : [[0.3876953  0.47808158 0.4033203  0.49761283 0.26342773]
 [0.6645508  0.48198783 0.6713867  0.49370658 0.2319336 ]]
2025-09-17 07:56:13,573 | INFO: Model original pred : [[0.4050293 0.4592014 0.4362793 0.4956597 0.3474121]]
Files: 100%|██████████| 3/3 [00:20<00:00,  6.71s/it]

Saved updated file to data/demo77/moret-sur-loing_-_280-13-22/response.json
